## Day 28 Lecture 2 Assignment

In this assignment, we will learn about overfitting and regularization. We will use the king county housing dataset loaded below and analyze the regression from this dataset.

In [0]:
%matplotlib inline

import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from statsmodels.tools.eval_measures import mse, rmse

In [0]:
king_county = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/kc_house_data.csv')

In [5]:
king_county.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


Perform the same transformations in the previous assignment to meet model assumptions:
1. Transform price and sqft_living by applying the natual log
2. Remove all columns except: log price, bedrooms, bathrooms, log_sqft_living, floors, waterfront
3. Split the data into train and test subsets. 20% of the data should be in the test subset

In [6]:
# answer below:
king_county['tot_sqft'] =  king_county['sqft_living'] + king_county['sqft_lot'] + king_county['sqft_above'] + king_county['sqft_basement'] + king_county['sqft_living15'] + king_county['sqft_lot15']
king_county['log_tot_sqft'] = np.log(king_county['tot_sqft'])
king_county.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,tot_sqft,log_tot_sqft
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,15000,9.615805
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,21711,9.985574
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062,22322,10.013328
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,15280,9.634300
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,20743,9.939964


In [7]:
king_county_f = king_county.drop(['id','date','view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'sqft_lot', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15', 'tot_sqft',
       'log_tot_sqft'], axis = 1)
king_county_f.head()

,price,bedrooms,bathrooms,sqft_living,floors,waterfront
0,221900.0,3,1.00,1180,1.0,0
1,538000.0,3,2.25,2570,2.0,0
2,180000.0,2,1.00,770,1.0,0
3,604000.0,4,3.00,1960,1.0,0
4,510000.0,3,2.00,1680,1.0,0


In [0]:
# answer below:
from sklearn.model_selection import train_test_split

X = king_county_f[['bedrooms', 'bathrooms', 'sqft_living', 'floors','waterfront']]
y = king_county_f['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=25)

Apply a ridge regression model with lambda=50 to the data and evaluate by looking at r squared for test and train

In [11]:
from sklearn.linear_model import Ridge

# Fitting a ridge regression model. Alpha is the regularization
# parameter (usually called lambda). As alpha gets larger, parameter
# shrinkage grows more pronounced.
ridgeregr = Ridge(alpha=50) 
ridgeregr.fit(X_train, y_train)

# We are making predictions here
y_preds_train = ridgeregr.predict(X_train)
y_preds_test = ridgeregr.predict(X_test)



print("R-squared of the model on the training set is: {}".format(ridgeregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model on the test set is: {}".format(ridgeregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model on the training set is: 0.5403429961022714
-----Test set statistics-----
R-squared of the model on the test set is: 0.5318522540390949
Mean absolute error of the prediction is: 169952.72996771097
Mean squared error of the prediction is: 66905376059.732185
Root mean squared error of the prediction is: 258660.73544264925
Mean absolute percentage error of the prediction is: 34.48225663841248


Perform a grid search for the following values of alpha: 1, 5, 10, 100, 500, 1000, 10000 to find the most optimal ridge regression model.

In [27]:
#answer below:

from sklearn.model_selection import GridSearchCV

tuned_parameters = [{'alpha': [1, 5, 10, 100, 500, 1000, 10000]}]

clf = GridSearchCV(Ridge(), param_grid=tuned_parameters, cv=5)
clf.fit(X_train, y_train)

print(clf.best_score_)
print(clf.best_estimator_.alpha)

0.5403750497694632
1
